# Visualizing Neuropixel Probe Locations

**Note: This notebook currently doesn't work on remote locations. To run it, it should be downloaded and run locally. Instructions for this can be found on the [Main Page](../intro.md)**

It can be handy to know the location and trajectory of the probes that obtain the data. Some NWB files have an **electrodes** field which store these locations. To be more precise, they contain arrays of *CCF* coordinates. [CCF](https://community.brain-map.org/t/allen-mouse-ccf-accessing-and-using-related-data-and-tools/359) is a framework which represents locations in the brain with coordinates that are relative to brain structure. This notebook uses CCF coordinate data in an extracellular electrophysiology NWB file to render the locations of the [Neuropixel](https://www.neuropixels.org/probe) probes that were used.

To be able to render locations in a brain, you don't need to know how CCF works except that it is a system of coordinates. We use **[ccf-widget](https://github.com/NeurodataWithoutBorders/ccf-widget)** as the library by which we 3D render the brain and the probe coordinates.

### Environment Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ccfwidget import CCFWidget
from dandi import dandiapi
from pynwb import NWBHDF5IO

### Downloading an NWB File
If you don't already have a file to analyze, you can use a file from The Allen Institute's **Visual Coding - Neuropixels** dataset. If you want to choose your own file to download, set `dandiset_id` and `dandi_filepath` accordingly.

In [2]:
dandiset_id = "000021"
dandi_filepath = "sub-703279277/sub-703279277_ses-719161530.nwb"
download_loc = "."

In [3]:
filename = dandi_filepath.split("/")[-1]
filepath = f"{download_loc}/{filename}"

In [4]:
my_dandiset = dandiapi.DandiAPIClient().get_dandiset(dandiset_id)
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(filepath)

print(f"Downloaded file to {filepath}")

A newer version (0.48.1) of dandi/dandi-cli is available. You are using 0.46.3


Downloaded file to ./sub-703279277_ses-719161530.nwb


### Extracting Electrode Data
Below the NWB file is read and the `electrodes` table is displayed. This table contains the metadata about the electrode channels along the Neuropixel probes used in this session. Importantly, this table contains the CCF coordinates, the brain location, and the name of the probe it belongs to.

In [5]:
### read the nwb file
io = NWBHDF5IO(filepath, mode="r", load_namespaces=True)
nwb = io.read()

Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.5.1 is already loaded.
Ignoring cached namespace 'core' version 2.2.2 because version 2.5.0 is already loaded.


In [6]:
nwb.electrodes[:]

,x,y,z,imp,location,filtering,group,group_name,probe_vertical_position,probe_horizontal_position,probe_id,local_index,valid_data
id,,,,,,,,,,,,,
850249265,7955.0,3766.0,3766.0,NaN,Eth,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeB abc.EcephysElectrodeGroup at 0x24370595...,probeB,20,43,729445650,0,False
850249267,7955.0,3756.0,3756.0,NaN,TH,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeB abc.EcephysElectrodeGroup at 0x24370595...,probeB,20,11,729445650,1,True
850249273,7955.0,3727.0,3727.0,NaN,TH,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeB abc.EcephysElectrodeGroup at 0x24370595...,probeB,60,43,729445650,4,True
850249277,7954.0,3708.0,3708.0,NaN,APN,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeB abc.EcephysElectrodeGroup at 0x24370595...,probeB,80,59,729445650,6,True
850249283,7954.0,3679.0,3679.0,NaN,APN,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeB abc.EcephysElectrodeGroup at 0x24370595...,probeB,100,11,729445650,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
850256499,7562.0,1193.0,1193.0,NaN,VISrl,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeF abc.EcephysElectrodeGroup at 0x24370595...,probeF,1240,27,729445658,123,True
850256501,7560.0,1186.0,1186.0,NaN,VISrl,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeF abc.EcephysElectrodeGroup at 0x24370595...,probeF,1260,43,729445658,124,True
850256505,7557.0,1172.0,1172.0,NaN,VISrl,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,probeF abc.EcephysElectrodeGroup at 0x24370595...,probeF,1280,59,729445658,126,True


### Getting CCF Coordinates
Here, the CCF coordinates are taken from the `electrodes` table and turned into an array of points.

In [7]:
### read the x,y,z ccf coordinates and generate points
xs = nwb.electrodes.x
ys = nwb.electrodes.y
zs = nwb.electrodes.z
n = min(len(xs), len(ys), len(zs))
# points = np.array([[xs[i], ys[i], zs[i]] for i in range(n)])
points = np.array(list(zip(xs,ys,zs)))

In [8]:
print(points.shape)

(2304, 3)


### Identifying Brain Regions
Since the brain location of each electrode is also stored, we can create a more informative display by coloring the points based on their location.

In [9]:
# assign each brain location a color
locations = list(set(nwb.electrodes.location))
colors = plt.cm.rainbow(np.linspace(0,1,len(locations)))
loc_color_map = {locations[i]:colors[i] for i in range(len(locations))}

# determine point colors depending on brain location
point_colors = [ loc_color_map[electrode] for electrode in nwb.electrodes.location]

### Rendering
Rendering is as simple as generating the widget and displaying it. This will create embedded window with the interactive 3D rendering of your scene. 

In [10]:
ccf = CCFWidget(markers=[points])

In [11]:
ccf

CCFWidget(children=(VBox(children=(Viewer(background=(0.85, 0.85, 0.85), camera=array([[ 1.3441567e+03, -2.172…